In [1]:
from torchvision import datasets, transforms
import math
import torch
import torch.nn.functional as nnf
import torch.nn as nn
import os
import time
import torch.optim as optim 
import datetime
import csv

#### Model

In [2]:
# get a pretrained network for transfer learning
import torchvision.models as models
densenet = models.densenet161(pretrained=True)

class transfer_music_classifer(nn.Module):
  def __init__(self):
    super(transfer_music_classifer, self).__init__()
    self.featureExtract = densenet.features
    
    self.classifier = nn.Sequential(
      nn.Linear(2208, 780),
      nn.ReLU(),
      nn.Dropout(0.2),
      nn.Linear(780, 240),
      nn.ReLU(),
      nn.Dropout(0.2),
      nn.Linear(240, 5),
      nn.Sigmoid(),
    )

  def forward(self, spectrogram):
    features = self.featureExtract(spectrogram)

    ######
    # This code is normally built into densenet, but since we are splitting
    # the features and classifier it must be put here
    # Taken from here: 
    # https://pytorch.org/vision/stable/_modules/torchvision/models/densenet.html

    # inplace = True means it modifies the input instead of allocating memory
    # for an ouput. Saves on memory
    out = nnf.relu(features, inplace=True)

    # adaptive pool decides the stride and kernel size automatically to ensure
    # the output has shape (x,x,1,1) regardless of input
    out = nnf.adaptive_avg_pool2d(out, (1, 1))

    # reshape output to be (1, x)
    out = torch.flatten(out, 1)
    ######    

    out = self.classifier(out)
    return out

#### Data loading

In [4]:
torch.manual_seed(1000)

# root dir for dataset !!! CHANGE TO OWN !!!
root_path = r'C:\Users\sjd30\Docnosync\spectrograms2'
#root_path = r'C:\Users\sjd30\Docnosync\overfit'

mytransforms = transforms.Compose([
  transforms.Resize([224,224]),
#   transforms.Grayscale(num_output_channels=1),
  transforms.ToTensor(),
  # densenet expects these transformations
  transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# load the dataset & labels from the given folder. Convert them to tensors
dataset = datasets.ImageFolder(root_path, transform=mytransforms)
print("Total num images:", len(dataset))

num_train_samples = math.floor(len(dataset)*0.7)
num_val_samples = math.floor(len(dataset)*0.15)
num_test_samples = math.floor(len(dataset)*0.15)

# add any extra images to the test set to avoid imperfect splitting
num_test_samples += len(dataset) - (num_train_samples + num_val_samples + num_test_samples)
print("number of images in train,val,test,total: ", num_train_samples, num_val_samples, num_test_samples, num_train_samples + num_val_samples + num_test_samples)

# uses pytorch method to randomly divide the loaded dataset with a set seed for reproducability
train_data, val_data, test_data = torch.utils.data.random_split(dataset, [num_train_samples, num_val_samples, num_test_samples], generator=torch.Generator().manual_seed(100))

Total num images: 108738
number of images in train,val,test,total:  76116 16310 16312 108738


#### Functions

In [5]:
# model is the network
# train_data and val_data are pytorch datasets
# batch_size, learning_rate, and num_epochs are hyperparameters
# save_dir defines the directory in which models will be saved
# net_name makes it easy to find the saved model in drive
# weight_decay is a parameter used in adam optimizer to 

def train_net(model, train_data, val_data, batch_size=32, learning_rate=1e-3, num_epochs=5, save=False, save_dir=r'C:\Users\sjd30\Downloads', weight_decay=1e-2, net_name='default'):

  # CE Loss for multi-class
  criterion = nn.CrossEntropyLoss()

  # adam optimizer for rapid training
  # the pretrained features of densenet are trained at 1/3 the learning rate
  # for fine tuning purposes.
  optimizer = optim.AdamW([
                {'params': model.featureExtract.parameters(), 'lr': learning_rate / 3.0},
                {'params': model.classifier.parameters() }
            ], lr=learning_rate, weight_decay = weight_decay)

  trainacc, iterations, lossrecords = [], [], []

  # create the dataloaders from the passed data
  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                          num_workers=0, shuffle=True)
  val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, 
                                            num_workers=0, shuffle=True)
  batch_num = 0
  start=time.time()
  for epoch in range(num_epochs):
    mini_batch_correct = 0
    mini_batch_total = 0
    #loops through each minibatch
    for spectrograms, labels in iter(train_loader):
      #Checks for GPU availability. Uses GPU if available.
      if torch.cuda.is_available():
        spectrograms = spectrograms.cuda()
        labels = labels.cuda()

      # standard training loop for pytroch NN.
      # gets outputs for the batch, computes losses,
      # uses backpropagation to compute the gradient
      # adjusts the weights, and cleans up the gradient
      optimizer.zero_grad()
      out = model(spectrograms)

      loss = criterion(out, labels)
      loss.backward()
      optimizer.step()
      
    
      pred = out.max(1, keepdim=True)[1]
      mini_batch_correct = pred.eq(labels.view_as(pred)).sum().item()
      mini_batch_total = spectrograms.shape[0]

      lossrecords.append(float(loss) / batch_size) # convert to float
      trainacc.append((mini_batch_correct / mini_batch_total))
      iterations.append(batch_num)
    
      # every 50 iteration give basic progress info
      if batch_num % 50 == 0:
          print('Batch Number {0}  Total Time (s): {1}  Progress: {2}%  Trainacc: {3}'.format(batch_num, time.time()-start, (100 * batch_num * batch_size) / (num_epochs * len(train_loader) * batch_size), trainacc[-1]))
      # progress to next batch
      batch_num += 1

    # saves the file with a descriptive name, including a timestamp to help
    # prevent overwriting files in case the net_name was not changed
    # also, since the model is large (100mb), dont save it unless it has
    # improved since the last epoch. -2 fixes a possible issue where the last
    # minibatch has skewed accuracy due to a different size
    # https://piazza.com/class/ko91w5f8es3619?cid=70
    if save:
        print('saving model')
        save_file = os.path.join(save_dir,'name-{0}_epoch-{1}_bs-{2}_lr-{3}_time-{4}'.format(net_name, epoch+1, batch_size, learning_rate, datetime.datetime.now().strftime("%H-%M-%S")))
        torch.save(model.state_dict(), save_file)
        
    valacc = eval_model(model, val_loader)
    
    csvPath = r'C:\Users\sjd30\Downloads\records.csv'
    newRow = [lossrecords[-1], trainacc[-1], iterations[-1], valacc]
    with open(csvPath, 'a') as csv:
        writer = csv.writer(csv)
        writer.writerow(newRow)
    
    # each epoch, give more detailed accuracy information
    print('Epoch number: {0}  Train acc: {1}  Val acc: {2}   Loss{3}\nTotal Time (s): {4}   Time per Epoch (s): {5}  \n\n'.format(
        epoch+1, 'n/a', valacc, lossrecords[-1], time.time() - start, (time.time() - start) / (1 + epoch) ))

In [6]:
def eval_model(model, loader_to_evaluate):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
      for spectrograms, labels in loader_to_evaluate:
        # use GPU
        if torch.cuda.is_available():
          spectrograms = spectrograms.cuda()
          labels = labels.cuda()

        # the output from model(spectrograms) is the probabilities of each genre
        # as the rows, and the classes as the columns. We find the max
        # probability class (dim = 1) and that returns a 2d tensor with only
        # columns. To make this a 1d tensor, we take the dim 1 (like squeezing)
        classPrediction = model(spectrograms).max(1, keepdim=True)[1]
        # adds up all of the  times that the label equal the prediction. Taken
        # from tut3a notebook
        correct += classPrediction.eq(labels.view_as(classPrediction)).sum().item()
        total += spectrograms.shape[0]
  model.train()
  return correct / total

#### Training

In [6]:
model = transfer_music_classifer()
#state = torch.load(r'C:\Users\sjd30\Downloads\name-low_weight_decay_and_lr_epoch-22_bs-16_lr-0.0002_time-07-27-49')
#model.load_state_dict(state)

# Train on GPU
if torch.cuda.is_available():
    print('Using GPU')
    model.cuda()
else:
    print('Using CPU, consider using a GPU runtime to speed up training')
    
train_net(model, train_data, val_data, batch_size=16, num_epochs=30, learning_rate = 5e-4, save=True, net_name = 'low_weight_decay_and_lr_saturday', weight_decay = 1e-2)

Using GPU


c:\users\sjd30\appdata\local\programs\python\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Batch Number 0  Total Time (s): 2.430799961090088  Progress: 0.0%  Trainacc: 0.8125
Batch Number 50  Total Time (s): 18.039762258529663  Progress: 0.03502872355331372%  Trainacc: 0.8125
Batch Number 100  Total Time (s): 33.652745962142944  Progress: 0.07005744710662744%  Trainacc: 0.9375
Batch Number 150  Total Time (s): 49.5377459526062  Progress: 0.10508617065994115%  Trainacc: 0.75
Batch Number 200  Total Time (s): 65.51781344413757  Progress: 0.14011489421325488%  Trainacc: 0.9375
Batch Number 250  Total Time (s): 82.09575319290161  Progress: 0.1751436177665686%  Trainacc: 0.875
Batch Number 300  Total Time (s): 98.55505156517029  Progress: 0.2101723413198823%  Trainacc: 0.9375
Batch Number 350  Total Time (s): 115.10901427268982  Progress: 0.245201064873196%  Trainacc: 0.875
Batch Number 400  Total Time (s): 131.0127453804016  Progress: 0.28022978842650975%  Trainacc: 0.8125
Batch Number 450  Total Time (s): 146.75038719177246  Progress: 0.31525851197982346%  Trainacc: 0.8125
Batc

Batch Number 4100  Total Time (s): 1321.6037483215332  Progress: 2.872355331371725%  Trainacc: 0.75
Batch Number 4150  Total Time (s): 1337.406681060791  Progress: 2.9073840549250387%  Trainacc: 0.875
Batch Number 4200  Total Time (s): 1353.290761232376  Progress: 2.942412778478352%  Trainacc: 0.875
Batch Number 4250  Total Time (s): 1369.0737552642822  Progress: 2.977441502031666%  Trainacc: 0.9375
Batch Number 4300  Total Time (s): 1384.8826112747192  Progress: 3.0124702255849796%  Trainacc: 0.75
Batch Number 4350  Total Time (s): 1400.7097446918488  Progress: 3.0474989491382933%  Trainacc: 0.9375
Batch Number 4400  Total Time (s): 1416.5327475070953  Progress: 3.082527672691607%  Trainacc: 0.75
Batch Number 4450  Total Time (s): 1432.3097581863403  Progress: 3.117556396244921%  Trainacc: 0.875
Batch Number 4500  Total Time (s): 1448.1047468185425  Progress: 3.1525851197982346%  Trainacc: 0.9375
Batch Number 4550  Total Time (s): 1463.8787548542023  Progress: 3.1876138433515484%  Tra

Batch Number 8100  Total Time (s): 2697.7411437034607  Progress: 5.674653215636822%  Trainacc: 0.875
Batch Number 8150  Total Time (s): 2713.385752439499  Progress: 5.709681939190136%  Trainacc: 0.9375
Batch Number 8200  Total Time (s): 2729.0688676834106  Progress: 5.74471066274345%  Trainacc: 0.6875
Batch Number 8250  Total Time (s): 2744.7247548103333  Progress: 5.779739386296764%  Trainacc: 0.875
Batch Number 8300  Total Time (s): 2760.369759082794  Progress: 5.814768109850077%  Trainacc: 0.9375
Batch Number 8350  Total Time (s): 2776.051746606827  Progress: 5.849796833403391%  Trainacc: 1.0
Batch Number 8400  Total Time (s): 2791.820754289627  Progress: 5.884825556956704%  Trainacc: 0.8125
Batch Number 8450  Total Time (s): 2807.5579917430878  Progress: 5.919854280510018%  Trainacc: 0.875
Batch Number 8500  Total Time (s): 2823.2622241973877  Progress: 5.954883004063332%  Trainacc: 0.875
Batch Number 8550  Total Time (s): 2839.0297527313232  Progress: 5.989911727616645%  Trainacc:

...

Batch Number 111300  Total Time (s): 37538.27275514603  Progress: 77.97393862967634%  Trainacc: 0.9375
Batch Number 111350  Total Time (s): 37553.71954870224  Progress: 78.00896735322965%  Trainacc: 0.8125
Batch Number 111400  Total Time (s): 37569.209587812424  Progress: 78.04399607678296%  Trainacc: 0.9375
Batch Number 111450  Total Time (s): 37584.73812127113  Progress: 78.07902480033627%  Trainacc: 0.9375
Batch Number 111500  Total Time (s): 37600.42153549194  Progress: 78.1140535238896%  Trainacc: 0.875
Batch Number 111550  Total Time (s): 37615.94800400734  Progress: 78.1490822474429%  Trainacc: 0.875
Batch Number 111600  Total Time (s): 37631.4927546978  Progress: 78.18411097099622%  Trainacc: 1.0
Batch Number 111650  Total Time (s): 37647.106541872025  Progress: 78.21913969454953%  Trainacc: 0.9375
Batch Number 111700  Total Time (s): 37662.69564604759  Progress: 78.25416841810285%  Trainacc: 0.75
Batch Number 111750  Total Time (s): 37678.23197579384  Progress: 78.289197141656

Batch Number 115250  Total Time (s): 38897.95987844467  Progress: 80.74120779038812%  Trainacc: 1.0
Batch Number 115300  Total Time (s): 38913.5248811245  Progress: 80.77623651394143%  Trainacc: 0.8125
Batch Number 115350  Total Time (s): 38929.0287528038  Progress: 80.81126523749475%  Trainacc: 0.9375
Batch Number 115400  Total Time (s): 38944.48380994797  Progress: 80.84629396104806%  Trainacc: 0.875
Batch Number 115450  Total Time (s): 38959.95389175415  Progress: 80.88132268460137%  Trainacc: 1.0
Batch Number 115500  Total Time (s): 38975.445758104324  Progress: 80.91635140815468%  Trainacc: 0.9375
Batch Number 115550  Total Time (s): 38990.92560362816  Progress: 80.951380131708%  Trainacc: 0.875
Batch Number 115600  Total Time (s): 39006.43581199646  Progress: 80.98640885526132%  Trainacc: 1.0
Batch Number 115650  Total Time (s): 39021.885759592056  Progress: 81.02143757881463%  Trainacc: 0.75
Batch Number 115700  Total Time (s): 39037.463024139404  Progress: 81.05646630236794%  T

Batch Number 119200  Total Time (s): 40253.011724472046  Progress: 83.50847695109991%  Trainacc: 0.8125
Batch Number 119250  Total Time (s): 40268.506009578705  Progress: 83.54350567465322%  Trainacc: 1.0
Batch Number 119300  Total Time (s): 40283.92502307892  Progress: 83.57853439820653%  Trainacc: 0.8125
Batch Number 119350  Total Time (s): 40299.36801838875  Progress: 83.61356312175984%  Trainacc: 1.0
Batch Number 119400  Total Time (s): 40314.8317425251  Progress: 83.64859184531316%  Trainacc: 0.9375
Batch Number 119450  Total Time (s): 40330.29375767708  Progress: 83.68362056886647%  Trainacc: 0.9375
Batch Number 119500  Total Time (s): 40345.70400452614  Progress: 83.71864929241978%  Trainacc: 1.0
Batch Number 119550  Total Time (s): 40361.22175216675  Progress: 83.7536780159731%  Trainacc: 0.8125
Batch Number 119600  Total Time (s): 40376.69800019264  Progress: 83.78870673952642%  Trainacc: 0.9375
Batch Number 119650  Total Time (s): 40392.130761146545  Progress: 83.823735463079

Batch Number 123250  Total Time (s): 41512.93966126442  Progress: 86.34580355891832%  Trainacc: 0.875
Batch Number 123300  Total Time (s): 41528.75975394249  Progress: 86.38083228247163%  Trainacc: 0.875
Batch Number 123350  Total Time (s): 41544.547755002975  Progress: 86.41586100602494%  Trainacc: 0.875
Batch Number 123400  Total Time (s): 41560.385514736176  Progress: 86.45088972957825%  Trainacc: 0.8125
Batch Number 123450  Total Time (s): 41576.11920237541  Progress: 86.48591845313157%  Trainacc: 0.8125
Batch Number 123500  Total Time (s): 41591.85852241516  Progress: 86.52094717668488%  Trainacc: 0.6875
Batch Number 123550  Total Time (s): 41607.65200853348  Progress: 86.5559759002382%  Trainacc: 0.75
Batch Number 123600  Total Time (s): 41623.403749227524  Progress: 86.5910046237915%  Trainacc: 0.875
Batch Number 123650  Total Time (s): 41639.167424201965  Progress: 86.62603334734483%  Trainacc: 0.9375
Batch Number 123700  Total Time (s): 41654.89901661873  Progress: 86.66106207

KeyboardInterrupt: 

The stuff below here is probably bad or otherwise unecesary


In [ ]:
# this is for testing purposes only
import glob
from PIL import Image
import matplotlib.pyplot as plt

images=glob.glob("/content/149244013_c529578289.jpg")
for image in images:
  img = Image.open(image).convert('L')
  trans = transforms.ToPILImage()
  trans1 = transforms.ToTensor()
  tens = trans(trans1(img))
  plt.imshow(tens)

  model = transfer_music_classifer()
  spectrogram = trans1(img).unsqueeze(0)
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay = 1e-5)

  print( model(spectrogram) )
  optimizer.zero_grad()
  out = model(spectrogram)
  loss = criterion(out, torch.empty(1, dtype=torch.long).random_(5))
  loss.backward()
  print(optimizer.step())


In [ ]:
# dont include in final report because i literally stole this from here https://stackoverflow.com/questions/49201236/check-the-total-number-of-parameters-in-a-pytorch-model
# this just counts parameters
from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

net = transfer_music_classifer()
count_parameters(net)

+--------------------------------------------------------+------------+
|                        Modules                         | Parameters |
+--------------------------------------------------------+------------+
|                featureExtract.0.weight                 |     27     |
|                 featureExtract.0.bias                  |     3      |
|             featureExtract.1.conv0.weight              |   14112    |
|             featureExtract.1.norm0.weight              |     96     |
|              featureExtract.1.norm0.bias               |     96     |
| featureExtract.1.denseblock1.denselayer1.norm1.weight  |     96     |
|  featureExtract.1.denseblock1.denselayer1.norm1.bias   |     96     |
| featureExtract.1.denseblock1.denselayer1.conv1.weight  |   18432    |
| featureExtract.1.denseblock1.denselayer1.norm2.weight  |    192     |
|  featureExtract.1.denseblock1.denselayer1.norm2.bias   |    192     |
| featureExtract.1.denseblock1.denselayer1.conv2.weight  |   829

26489702